In [ ]:
import os
from google.colab import drive
drive.mount('/gdrive')
os.symlink('/gdrive/My Drive', '/content/gdrive')

os.chdir("gdrive/ml")

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


FileExistsError: ignored

In [ ]:
import numpy as np
import math
from random import *
class TreeNode:
    def __init__(self,index,v):
        self.index=index
        self.val = v
        self.sign=0
        self.left=None
        self.right=None
def read(path):
    temp=np.loadtxt(path)
    c=np.delete(temp, (10), axis=1)
    return c,temp[:,10]

def dstump(x,y):
    thetas=np.array([ (x[i]+x[i+1])/2 for i in range(0, x.shape[0]-1) ] )
    np.append(thetas,x[0]-1)
    np.append(thetas,x[x.shape[0]-1]+1)
    B = float("inf")
    target_theta = 0.0
    for theta in thetas:
        ly = y[np.where(x<theta)]
        ry = y[np.where(x>=theta)]
        b = ly.shape[0]*gini(ly) + ry.shape[0]*gini(ry)
        if B>b:
            B = b
            target_theta = theta
    return B, target_theta

def split(x,y):
    sort=[]
    for i in range(0, x.shape[1]):
        sort.append(np.argsort(x[:,i]))
    Branch = float("inf")
    index = -1
    val = 0
    for i in range(0, x.shape[1]):
        xi = x[sort[i], i]
        yi = y[sort[i]]
        b,v = dstump(xi, yi)
        if Branch>b:
            Branch = b
            index = i
            val = v
    leftX = x[np.where(x[:,index]<val)]
    leftY = y[np.where(x[:,index]<val)]
    rightX = x[np.where(x[:,index]>=val)]
    rightY = y[np.where(x[:,index]>=val)]
    return leftX, leftY, rightX, rightY, index, val
        
def gini(y):
    if y.shape[0]==0: return 0
    n1 = sum(y==1)
    n2 = sum(y==-1)
    if (n1+n2)==0: return 0
    return 1.0 - math.pow(1.0*n1/(n1+n2),2) - math.pow(1.0*n2/(n1+n2),2)

def cart(x, y):
    if x.shape[0]==0: return None 
    if gini(y)==0:
        node = TreeNode(-1, -1)
        node.sign = 1 if y[0]==1 else -1
        return node
    leftX, leftY, rightX, rightY, index, val = split(x,y)
    node = TreeNode(index,val)
    node.left = cart(leftX, leftY)
    node.right = cart(rightX, rightY)
    return node
def predict(root, x):
    if root.index==-1: return root.sign
    if x[root.index]<root.val:
        return predict(root.left, x)
    else:
        return predict(root.right, x)
def eout(x,y,root):
    count=0
    for i in range(x.shape[0]):
        if(y[i]!=predict(root,x[i])):
            count+=1
    return count/x.shape[0]
    
x,y=read("hw6_train.dat.txt")
root=cart(x,y)
tx,ty=read("hw6_test.dat.txt")
print(eout(tx,ty,root))

0.166


In [ ]:
import random
l=[random.randint(0, 999) for _ in range(500)]
sampleX = x[l]
sampleY = y[l]
print(sampleX)

[[ 3.36166084 -0.62896605  0.33367441 ... -0.3545793   0.03107955
   0.41800037]
 [ 0.06191806  2.26826144 -0.19490531 ...  0.38769975  0.13029247
  -0.80054501]
 [ 3.11390273  1.91854494 -1.47927738 ... -0.32422422  1.35692795
  -0.31938439]
 ...
 [ 5.76511659  1.75464735  1.12796994 ... -1.00259931 -1.33884238
   1.3065909 ]
 [ 2.36913286  0.1227638  -1.73365708 ...  0.18406491 -0.16235588
  -0.44003895]
 [-1.84624402 -1.16138051 -1.56987723 ... -0.04162235  0.61791507
   0.60463548]]


In [ ]:
import random

In [ ]:
def randomForest(x, y, T):
    trees = []
    oob=np.zeros((1000,2000))
    for i in range(0,T):
        if(i%10==0):
            print(i)
        xi,yi,oo = sampling(x, y)
        for j in oo:
            oob[j][i]=1    
        root = cart(xi,yi)
        trees.append(root)
    return  trees,oob

def sampling(x, y):
    l=[random.randint(0, 999) for _ in range(500)]
    sampleX = x[l]
    sampleY = y[l]
    return sampleX, sampleY,l
trees,oob=randomForest(x,y,2000)
seout=0
j=0
tx,ty=read("hw6_test.dat.txt")
for i in trees:
    j+=1
    if(j%100==0):
            print(j)
    seout+=eout(tx,ty,i)
print(seout/2000)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
0.2305


In [ ]:
def ein(x,y,trees):
    count=0 
    for i in range(1000):
        py=rfpred(x[i],trees)
        if(py!=y[i]):
            count+=1
    return count/1000
def rfpred(x,trees):
    res=0
    for i in trees:
        y=predict(i,x)
        res+=y
    return 1 if res>0 else -1

print(ein(x,y,trees))
print(ein(tx,ty,trees))

0.013
0.159


In [ ]:
eoob=0
o=np.zeros(1000)
for i in range(1000):
    for j in range(2000):
        if(oob[i][j]!=1):
            o[i]+=predict(trees[j],x[i])

for i in range(1000):
    if(o[i]*y[i]<0):
        eoob+=1
    
print(eoob/1000)

0.074


In [ ]:
import numpy as np
import math
import random
def read(path):
    temp=np.loadtxt(path)
    c=np.delete(temp, (10), axis=1)
    return c,temp[:,10]
x,y=read("hw6_train.dat.txt")

In [ ]:
from sklearn.tree import DecisionTreeClassifier
def randomForest(x, y, T):
    trees = []
    oob=np.zeros((1000,2000))
    for i in range(0,T):
        if(i%10==0):
            print(i)
        xi,yi,oo = sampling(x, y)
        for j in oo:
            oob[j][i]=1 
        d=DecisionTreeClassifier()   
        root = d.fit(xi,yi)
        trees.append(root)
    return  trees,oob

def sampling(x, y):
    l=[random.randint(0, 999) for _ in range(500)]
    sampleX = x[l]
    sampleY = y[l]
    return sampleX, sampleY,l
trees,oob=randomForest(x,y,2000)
eoob=0
o=np.zeros(1000)
for i in range(1000):
    for j in range(2000):
        if(oob[i][j]!=1):
            o[i]+=trees[j].predict([x[i]])

for i in range(1000):
    if(o[i]*y[i]<0):
        eoob+=1
    
print(eoob/1000)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
0.0


In [ ]:
eoob=0
o=np.zeros(1000)
for i in range(1000):
    for j in range(2000):
        if(oob[i][j]!=1):
            o[i]+=trees[j].predict([x[i]])

for i in range(1000):
    if(o[i]*y[i]<0):
        eoob+=1
    
print(eoob/1000)

0.073
